<!--
SPDX-FileCopyrightText: Copyright (c) 2024-2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
-->

# Deploy HuggingFace Safetensor LLM Weights with NIM

This notebook shows you how to deploy the most common type of LLM - HuggingFace models in Safetensors format - using NVIDIA NIM. This is the easiest way to get started with NIM deployment. For the complete list of LLMs supported by NIM, see the [NIM LLM Documentation](https://docs.nvidia.com/nim/large-language-models/latest/supported-llm-agnostic-architectures.html)

## What You'll Build

By the end of this notebook, you'll be able to:
- ✅ Deploy any HuggingFace model with a single Docker command
- ✅ Switch between inference backends (TensorRT-LLM, vLLM) for different use cases
- ✅ Customize deployment parameters for your specific needs
- ✅ Deploy models offline from local storage

## When to Use This Approach

**Choose this notebook if you:**
- Want to deploy popular models like Llama, Mistral, or Codestral
- Have models already available on HuggingFace
- Need a quick deployment without manual optimization
- Want to experiment with different inference backends

## How It Works

NIM makes deployment simple:
1. **You provide**: A HuggingFace model path (e.g., `mistralai/Codestral-22B-v0.1`)
2. **NIM automatically**:
   - Downloads the model
   - Analyzes the architecture
   - Selects the best backend
   - Optimizes for your GPU
   - Starts serving with an OpenAI-compatible API

## What's Covered

This tutorial includes:
* **Example 1**: Basic deployment from HuggingFace (5 minutes)
* **Example 2**: Choosing different backends (TensorRT-LLM vs vLLM)
* **Example 3**: Customizing model parameters
* **Example 4**: Deploying from local storage for offline use

## Prerequisites

### Hardware Requirements

Before proceeding, ensure your system meets the following requirements:

- **GPU**: NVIDIA GPU with at least 32GB VRAM (for Codestral-22B) or 8GB VRAM (for smaller models like Qwen2.5-0.5B)
- **System Memory**: At least 32GB RAM recommended
- **Storage**: Sufficient disk space for model downloads and caching

For detailed hardware specifications, please refer to the [NIM LLM Documentation](https://docs.nvidia.com/nim/large-language-models/latest/getting-started.html).

### System Setup

First, let's verify your GPU setup and install necessary dependencies:




In [ ]:
!nvidia-smi

### Install Required Software




In [ ]:
# Install Python dependencies
%pip install docker requests huggingface-hub && echo "✓ Python dependencies installed successfully"

### Get API Keys

#### NVIDIA NGC API Key

The NVIDIA NGC API Key is mandatory for accessing NVIDIA container registry and pulling secure container images.
Refer to [Generating NGC API Keys](https://docs.nvidia.com/ngc/gpu-cloud/ngc-user-guide/index.html#generating-api-key) for more information.




In [ ]:
import getpass
import os

if not os.environ.get("NGC_API_KEY", "").startswith("nvapi-"):
    ngc_api_key = getpass.getpass("Enter your NGC API Key: ")
    assert ngc_api_key.startswith("nvapi-"), "Not a valid key"
    os.environ["NGC_API_KEY"] = ngc_api_key
    print("✓ NGC API Key set successfully")

In [ ]:
!echo "${NGC_API_KEY}" | docker login nvcr.io -u '$oauthtoken' --password-stdin

#### Hugging Face Token

You'll also need a [Huggingface Token](https://huggingface.co/settings/tokens) to download models.




In [ ]:
if not os.environ.get("HF_TOKEN", "").startswith("hf_"):
    hf_token = getpass.getpass("Enter your Huggingface Token: ")
    assert hf_token.startswith("hf_"), "Not a valid key"
    os.environ["HF_TOKEN"] = hf_token
    print("✓ Hugging Face token set successfully")

### Setup NIM Container

Choose your NIM container image and pull it:




In [ ]:
# Set the NIM image
os.environ['NIM_IMAGE'] = "nvcr.io/nvidia/nim/nim-llm:1.11.0"
print(f"Using NIM image: {os.environ['NIM_IMAGE']}")

In [ ]:
# Pull the NIM container image
!docker pull $NIM_IMAGE && echo "✓ NIM container image pulled successfully"

### Utility Functions

Below are some utility functions we'll use in this notebook. These are for simplifying the process of deploying and monitoring NIMs in a notebook environment, and aren't required in general.



In [ ]:
import requests
import time
import docker
import os

def check_service_ready_from_logs(container_name, print_logs=False, timeout=600):
    """
    Check if NIM service is ready by monitoring Docker logs for 'Application startup complete' message.

    Args:
        container_name (str): Name of the Docker container
        print_logs (bool): Whether to print logs while monitoring (default: False)
        timeout (int): Maximum time to wait in seconds (default: 600)

    Returns:
        bool: True if service is ready, False if timeout reached
    """
    print("Waiting for NIM service to start...")
    start_time = time.time()

    try:
        client = docker.from_env()
        container = client.containers.get(container_name)

        # Stream logs in real-time using the blocking generator
        log_buffer = ""
        for log_chunk in container.logs(stdout=True, stderr=True, follow=True, stream=True):
            # Check timeout
            if time.time() - start_time > timeout:
                print(f"❌ Timeout reached ({timeout}s). Service may not have started properly.")
                return False

            # Decode chunk and add to buffer
            chunk = log_chunk.decode('utf-8', errors='ignore')
            log_buffer += chunk

            # Process complete lines
            while '\n' in log_buffer:
                line, log_buffer = log_buffer.split('\n', 1)
                line = line.strip()

                if print_logs and line:
                    print(f"[LOG] {line}")

                # Check for startup complete message
                if "Application startup complete" in line:
                    print("✓ Application startup complete! Service is ready.")
                    return True

    except Exception as e:
        print(f"❌ Error: {e}")
        return False

    print(f"❌ Timeout reached ({timeout}s). Service may not have started properly.")
    return False

def check_service_ready():
    """Fallback health check using HTTP endpoint"""
    url = 'http://localhost:8000/v1/health/ready'
    print("Checking service health endpoint...")

    while True:
        try:
            response = requests.get(url, headers={'accept': 'application/json'})
            if response.status_code == 200 and response.json().get("message") == "Service is ready.":
                print("✓ Service ready!")
                break
        except requests.ConnectionError:
            pass
        print("⏳ Still starting...")
        time.sleep(30)

def generate_text(model, prompt, max_tokens=1000, temperature=0.7):
    """Generate text using the NIM service"""
    try:
        response = requests.post(
            f"http://localhost:8000/v1/chat/completions",
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": max_tokens,
                "temperature": temperature
            },
            timeout=60
        )
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

print("✓ Utility functions loaded successfully")

## Deployment Examples

Let's explore different ways to deploy models using NIM.

### Example 1: Basic Deployment from Hugging Face

This example shows how to deploy Codestral-22B with default settings directly from Hugging Face.

<div class="alert alert-block alert-warning">
    <b>Note:</b> NVIDIA cannot guarantee the security of any models hosted on non-NVIDIA systems such as HuggingFace. Malicious or insecure models can result in serious security risks up to and including full remote code execution. We strongly recommend that before attempting to load it you manually verify the safety of any model not provided by NVIDIA, through such mechanisms as a) ensuring that the model weights are serialized using the safetensors format, b) conducting a manual review of any model or inference code to ensure that it is free of obfuscated or malicious code, and c) validating the signature of the model, if available, to ensure that it comes from a trusted source and has not been modified.
</div>

<div class="alert alert-block alert-info">
<b>Important:</b> You must accept the model's license agreement at https://huggingface.co/mistralai/Codestral-22B-v0.1 before using this model.
</div>




In [ ]:
# Set base directory for all files - you can modify this path as needed
# Examples: ".", "~", "/tmp", "/scratch", etc.
base_work_dir = "/ephemeral"
os.environ["BASE_WORK_DIR"] = base_work_dir

container_name = "LLM-NIM"
os.environ["CONTAINER_NAME"] = container_name
os.environ["LOCAL_NIM_CACHE"] = os.path.join(base_work_dir, ".cache/nim")
os.makedirs(os.environ["LOCAL_NIM_CACHE"], exist_ok=True)

In [ ]:
!docker run -it --rm \
 --name=$CONTAINER_NAME \
 --runtime=nvidia \
 --gpus all \
 --shm-size=16GB \
 -e HF_TOKEN=$HF_TOKEN \
 -e NIM_MODEL_NAME="hf://mistralai/Codestral-22B-v0.1" \
 -e NIM_SERVED_MODEL_NAME="mistralai/Codestral-22B-v0.1" \
 -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
 -u $(id -u) \
 -p 8000:8000 \
 -d \
 $NIM_IMAGE

After running the following cell, you should be able to see the `LLM-NIM` container running.





In [ ]:
!docker ps  # Check container is running

While the LLM NIM service is getting ready, you may run the following cell to see live logs.

<div class="alert alert-block alert-info">
<b>Note:</b> NIM service takes a few minutes to initialize. Monitor with logs if needed.
</div>




In [ ]:
check_service_ready_from_logs(os.environ["CONTAINER_NAME"], print_logs=True)

Now let's test the deployed model:




In [ ]:
result = generate_text(
    model="mistralai/Codestral-22B-v0.1",
    prompt="Write a complete function that computes fibonacci numbers in Rust"
)
print(result if result else "Failed to generate text")

Before we move onto the next example, let's stop the LLM NIM service.





In [ ]:
!docker stop $CONTAINER_NAME 2>/dev/null || echo "Container already stopped"

### Example 2: Deployment Using Different Backend Options

NIM supports multiple backends for model deployment. Let's explore TensorRT-LLM and vLLM backends:

#### TensorRT-LLM Backend




In [ ]:
# Using TensorRT-LLM backend by specifying the NIM_MODEL_PROFILE parameter
!docker run -it --rm \
 --name=$CONTAINER_NAME \
 --runtime=nvidia \
 --gpus all \
 --shm-size=16GB \
 -e HF_TOKEN=$HF_TOKEN \
 -e NIM_MODEL_NAME="hf://mistralai/Codestral-22B-v0.1" \
 -e NIM_SERVED_MODEL_NAME="mistralai/Codestral-22B-v0.1" \
 -e NIM_MODEL_PROFILE="tensorrt_llm" \
 -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
 -u $(id -u) \
 -p 8000:8000 \
 -d \
 $NIM_IMAGE

In [ ]:
check_service_ready_from_logs(os.environ["CONTAINER_NAME"], print_logs=True)

Test the TensorRT-LLM backend:




In [ ]:
result = generate_text(
    model="mistralai/Codestral-22B-v0.1",
    prompt="Write a complete Python function that computes fibonacci numbers with memoization"
)
print("TensorRT-LLM Backend Result:")
print("=" * 50)
print(result if result else "Failed to generate text")

Before we move onto the next example, let's stop the LLM NIM service.





In [ ]:
!docker stop $CONTAINER_NAME 2>/dev/null || echo "Container already stopped"

#### vLLM Backend




In [ ]:
# Using vLLM backend by specifying the NIM_MODEL_PROFILE parameter
!docker run -it --rm \
 --name=$CONTAINER_NAME \
 --runtime=nvidia \
 --gpus all \
 --shm-size=16GB \
 -e HF_TOKEN=$HF_TOKEN \
 -e NIM_MODEL_NAME="hf://mistralai/Codestral-22B-v0.1" \
 -e NIM_SERVED_MODEL_NAME="mistralai/Codestral-22B-v0.1" \
 -e NIM_MODEL_PROFILE="vllm" \
 -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
 -u $(id -u) \
 -p 8000:8000 \
 -d \
 $NIM_IMAGE

In [ ]:
check_service_ready_from_logs(os.environ["CONTAINER_NAME"], print_logs=True)

Test the vLLM backend:





In [ ]:
result = generate_text(
    model="mistralai/Codestral-22B-v0.1",
    prompt="Write a complete C++ function that computes fibonacci numbers efficiently"
)
print("vLLM Backend Result:")
print("=" * 50)
print(result if result else "Failed to generate text")

Before we move onto the next example, let's stop the LLM NIM service.





In [ ]:
!docker stop $CONTAINER_NAME 2>/dev/null || echo "Container already stopped"

### Example 3: Customizing Model Parameters

This example demonstrates how custom parameters affect model behavior. We'll deploy with specific constraints and test them:

**Key Parameters:**
* `NIM_TENSOR_PARALLEL_SIZE=2`: Uses 2 GPUs in parallel for better performance
* `NIM_MAX_INPUT_LENGTH=2048`: Limits input to 2048 tokens
* `NIM_MAX_OUTPUT_LENGTH=512`: Limits output to 512 tokens

<div class="alert alert-block alert-info">
    <b>Note:</b> You must have at least 2 GPUs to run the following cell. If you don't have at least 2 GPUs, modify the <code>NIM_TENSOR_PARALLEL_SIZE</code> paramater in the cell below.
</div>


In [ ]:
!docker run -it --rm \
 --name=$CONTAINER_NAME \
 --runtime=nvidia \
 --gpus all \
 --shm-size=16GB \
 -e HF_TOKEN=$HF_TOKEN \
 -e NIM_MODEL_NAME="hf://mistralai/Codestral-22B-v0.1" \
 -e NIM_SERVED_MODEL_NAME="mistralai/Codestral-22B-v0.1" \
 -e NIM_TENSOR_PARALLEL_SIZE=2 \
 -e NIM_MAX_INPUT_LENGTH=2048 \
 -e NIM_MAX_OUTPUT_LENGTH=512 \
 -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
 -u $(id -u) \
 -p 8000:8000 \
 -d \
 $NIM_IMAGE

In [ ]:
# Use the log-based check (set print_logs=True to see detailed logs)
check_service_ready_from_logs(os.environ["CONTAINER_NAME"], print_logs=True)

Test with custom parameters:





In [ ]:
result = generate_text(model="mistralai/Codestral-22B-v0.1",
                       prompt="Write me a function that computes fibonacci in Javascript")
print(result if result else "Failed to generate text")

Before we move onto the next example, let's stop the LLM NIM service.





In [ ]:
!docker stop $CONTAINER_NAME 2>/dev/null || echo "Container already stopped"

### Example 4: Deployment from Local Model

This example shows how to deploy Qwen2.5-0.5B from the locally downloaded model:

#### Download Model to Local Storage

We'll download Qwen2.5-0.5B, a lightweight LLM, for use in Example 4.

<div class="alert alert-block alert-info">
<b>Note:</b> You can modify the `model_save_location` variable below to use a different directory for storing downloaded models.
</div>




In [ ]:
# Set up local model directory
model_save_location = os.path.join(base_work_dir, "models")
local_model_name = "Qwen2.5-0.5B-Instruct"
local_model_path = os.path.join(model_save_location, local_model_name)
os.makedirs(local_model_path, exist_ok=True)

os.environ["LOCAL_MODEL_DIR"] = local_model_path

In [ ]:
!huggingface-cli download Qwen/Qwen2.5-0.5B-Instruct --local-dir "$LOCAL_MODEL_DIR" && echo "✓ Model downloaded successfully"

In [ ]:
# Verify model files exist
!ls -Rlh "$LOCAL_MODEL_DIR"

In [ ]:
!docker run -it --rm \
 --name=$CONTAINER_NAME \
 --runtime=nvidia \
 --gpus '"device=0"' \
 --shm-size=16GB \
 -e NIM_MODEL_NAME="/opt/models/local_model" \
 -e NIM_SERVED_MODEL_NAME="Qwen/Qwen2.5-0.5B" \
 -v "$LOCAL_MODEL_DIR:/opt/models/local_model" \
 -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
 -u $(id -u) \
 -p 8000:8000 \
 -d \
 $NIM_IMAGE

In [ ]:
# Use the log-based check (set print_logs=True to see detailed logs)
check_service_ready_from_logs(os.environ["CONTAINER_NAME"], print_logs=True)

Test the local model deployment:





In [ ]:
result = generate_text(model="Qwen/Qwen2.5-0.5B",
                       prompt="Tell me a story about a cat")
print(result if result else "Failed to generate text")

In [ ]:
# Final cleanup
!docker stop $CONTAINER_NAME 2>/dev/null || echo "Container already stopped"
print("✓ All containers stopped successfully")